With Homotopy continuation, try some higher dimension global optimization problems. 

In [ ]:
include("construct_lib.jl")
include("functions_eval.jl")
using DynamicPolynomials, MultivariatePolynomials, HomotopyContinuation, JSON


# Constants and Parameters
const d1, d2, ds = 2, 8, 1  # Degree range and step
const n, a, b = 2, 1, 1
const C = a / b  # Scaling constant
const delta, alph = 1/2, 6 / 10  # Sampling parameters